In [4]:
import pandas as pd
import numpy as np
import json
import re

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

from sklearn.neighbors import NearestNeighbors
from sklearn import neighbors
from sklearn.metrics import classification_report

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

In [7]:
with open('Data/reviews_Clothing_Shoes_and_Jewelry_5.json') as f:
    data = json.load(f)

In [8]:
df = pd.DataFrame.from_dict(data, orient='columns')

In [9]:
#df = df.sample(n=50000)
#df.to_csv('Data/reviews_50000.csv', encoding='utf-8', index=False)
#df = pd.read_csv('Data/reviews_50000.csv')
#helpful = df[df['helpful'] != '[0, 0]']
df

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0000031887,"[0, 0]",5.0,This is a great tutu and at a really great pri...,"02 12, 2011",A1KLRMWW2FWPL4,"Amazon Customer ""cameramom""",Great tutu- not cheaply made,1297468800
1,0000031887,"[0, 0]",5.0,I bought this for my 4 yr old daughter for dan...,"01 19, 2013",A2G5TCU2WDFZ65,Amazon Customer,Very Cute!!,1358553600
2,0000031887,"[0, 0]",5.0,What can I say... my daughters have it in oran...,"01 4, 2013",A1RLQXYNCMWRWN,Carola,I have buy more than one,1357257600
3,0000031887,"[0, 0]",5.0,"We bought several tutus at once, and they are ...","04 27, 2014",A8U3FAMSJVHS5,Caromcg,"Adorable, Sturdy",1398556800
4,0000031887,"[0, 0]",5.0,Thank you Halo Heaven great product for Little...,"03 15, 2014",A3GEOILWLK86XM,CJ,Grammy's Angels Love it,1394841600
5,0000031887,"[0, 0]",4.0,I received this today and I'm not a fan of it ...,"03 31, 2014",A27UF1MSF3DB2,"C-Lo ""Cynthia""",It's ok,1396224000
6,0000031887,"[0, 0]",5.0,Bought this as a backup to the regular ballet ...,"05 3, 2014",A16GFPNVF4Y816,design maven,Great for dress-up and for ballet practice,1399075200
7,0000031887,"[0, 0]",5.0,Great tutu for a great price. It isn't a &#34;...,"12 23, 2012",A2M2APVYIB2U6K,Jamie P.,Great value,1356220800
8,0000031887,"[0, 0]",4.0,"My daughter liked this, and it with her costum...","11 10, 2013",A1NJ71X3YPQNQ9,JBerger,Good,1384041600
9,0000031887,"[7, 8]",5.0,For what I paid for two tutus is unbeatable an...,"10 7, 2012",A3EERSWHAI6SO,"Jeffrey Hollingshead ""Jillian hollingshead""",WOW !! ..is all I have to say!,1349568000


In [10]:
count = df.groupby('asin', as_index=False).count()
mean = df.groupby('asin', as_index=False).mean()
df_merged = pd.merge(df, count, how='right', on=['asin'])

In [11]:
df_merged['totalReviewers'] = df_merged['reviewerID_y']
df_merged["overallScore"] = df_merged["overall_x"]
df_merged["summaryReview"] = df_merged["summary_x"]
# df_merged["textReview"] = df_merged[["reviewText_x","summary_x"]]

df_new = df_merged[['asin','summaryReview','overallScore',"totalReviewers"]]

df_merged = df_merged.sort_values(by='totalReviewers', ascending=False)
df_select = df_merged[df_merged.totalReviewers >= 10]

In [12]:
df_product_reviews = df_select.groupby('asin')['summaryReview'].apply(list)
df_product_reviews = pd.DataFrame(df_product_reviews)

In [89]:
df_mean = pd.merge(df_product_reviews, mean, on='asin', how='inner')
df_mean = df_mean[['asin', 'summaryReview', 'overall']]

In [90]:
regEx = re.compile('[^a-z]+')
def cleanReviews(reviewText):
    value = []
    for i in range(len(reviewText)):
        value.append(reviewText[i].lower())
        value[i] = regEx.sub(' ', value[i]).strip()
    return value

In [91]:
df_mean['summaryClean'] = df_mean['summaryReview'].apply(cleanReviews)
#df_mean = df_mean.drop_duplicates(['overall'], keep='last')

In [134]:
reviews = df_mean['summaryClean']
reviews = reviews.reset_index().summaryClean
countVector = CountVectorizer(max_features = 300, stop_words='english')

corpus = []
for i in range(len(reviews)):
    string = ' '.join(reviews[i])
    corpus.append(string)
transformedReviews = countVector.fit_transform(corpus)

dfReviews = pd.DataFrame(transformedReviews.A, columns=countVector.get_feature_names())
dfReviews = dfReviews.astype(int)

In [93]:
X = np.array(dfReviews)
tpercent = 0.75
tsize = int(np.floor(tpercent * len(dfReviews)))
dfReviews_train = X[:tsize]
dfReviews_test = X[tsize:]

lentrain = len(dfReviews_train)
lentest = len(dfReviews_test)

In [173]:
neighbor = NearestNeighbors(n_neighbors=4, algorithm='ball_tree').fit(dfReviews_train)

def calc_neighbors(element):
    a = neighbor.kneighbors([element])
    list_similar = a[1]
    
    first_similar = [item[0] for item in list_similar]
    first_similar = int(str(first_similar).strip('[]'))

    second_similar = [item[1] for item in list_similar]
    second_similar = int(str(second_similar).strip('[]'))
    
    third_similar = [item[2] for item in list_similar]
    third_similar = int(str(third_similar).strip('[]'))
    
    fourth_similar = [item[3] for item in list_similar]
    fourth_similar = int(str(fourth_similar).strip('[]'))
    
#     fifth_similar = [item[4] for item in list_similar]
#     fifth_similar = int(str(fifth_similar).strip('[]'))
    
    result = []
    list_similar = list_similar[0]
    for i in range(4):
        ob = {
            'asin': df_mean['asin'][list_similar[i]],
            'index': list_similar[i]
        }
        result.append(ob)
    return result

In [182]:
#calc_neighbors(dfReviews_test[0])
# samples = df_mean[lentrain:].sample(n=10).index
samples = reviews[lentrain:]
samples = samples.sample(n=10).index
results = []
for i in range(len(samples)):
    r = calc_neighbors(dfReviews_test[samples[i] - lentrain])
    results.append([df_mean['asin'][samples[i]], r])
results
# df_mean['index1'] = df_mean.index
# with open('./Data/data_json.json', 'w') as f:
#json.dumps(df_mean[lentrain:].sample(n=10).values.tolist(), indent=4)
# print(df_mean[lentrain:].sample(n=10).values)
# df_mean

[['B00GC1WPJ0',
  [{'asin': 'B008A01R72', 'index': 6543},
   {'asin': 'B0046LVHN8', 'index': 3868},
   {'asin': 'B0033IJDM2', 'index': 3073},
   {'asin': 'B003CJJYJO', 'index': 3279}]],
 ['B00B2PMF1Y',
  [{'asin': 'B0058SUXVK', 'index': 4799},
   {'asin': 'B003U3GOFO', 'index': 3560},
   {'asin': 'B001TJ3MLU', 'index': 2312},
   {'asin': 'B003VMUJG4', 'index': 3607}]],
 ['B008LY48SW',
  [{'asin': 'B003XQN73U', 'index': 3668},
   {'asin': 'B002W6Z15Y', 'index': 2941},
   {'asin': 'B0003DKV8K', 'index': 228},
   {'asin': 'B001OEICG0', 'index': 2229}]],
 ['B009RV1IW2',
  [{'asin': 'B0052GJLL6', 'index': 4645},
   {'asin': 'B0058Y99BO', 'index': 4825},
   {'asin': 'B001BEZ8E2', 'index': 1912},
   {'asin': 'B000783X5E', 'index': 308}]],
 ['B009FHIRC2',
  [{'asin': 'B007JEHD5A', 'index': 6051},
   {'asin': 'B001W2G9A0', 'index': 2376},
   {'asin': 'B0043D1COI', 'index': 3793},
   {'asin': 'B001BEZ8E2', 'index': 1912}]],
 ['B00E39JYUQ',
  [{'asin': 'B0075UNVWC', 'index': 5884},
   {'asin': 'B

In [178]:
df5_train_target = df_mean["overall"][:lentrain]
df5_test_target = df_mean["overall"][lentrain:]
df5_train_target = df5_train_target.astype(int)
df5_test_target = df5_test_target.astype(int)

n_neighbors = 4
knnclf = neighbors.KNeighborsClassifier(n_neighbors, weights='distance')
knnclf.fit(dfReviews_train, df5_train_target)
knnpreds_test = knnclf.predict(dfReviews_test)

print(classification_report(df5_test_target, knnpreds_test))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         1
           2       0.50      0.05      0.09        40
           3       0.41      0.20      0.27       567
           4       0.75      0.91      0.82      1568
           5       0.00      0.00      0.00        10

   micro avg       0.70      0.70      0.70      2186
   macro avg       0.33      0.23      0.24      2186
weighted avg       0.65      0.70      0.66      2186



/Users/porquin/Documents/Jailson/env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [179]:
print(accuracy_score(df5_test_target, knnpreds_test))

0.7026532479414456


In [180]:
print(mean_squared_error(df5_test_target, knnpreds_test))

0.32570905763952424
